In [1]:
import pandas as pd
import json
import os

# Background

Post-scraping we have 2 types of data: basic restaurant details & subsequent review data. Let's clean and flatten the subsquent data frames so we can export them out into re-usable CSVs.

# Restaurant Reviews Data

In [2]:
def load_reviews(file_name): 
    '''
    load restaurant reviews data into pandas DataFrame from json file,
    flatten the nested structure to retrieve all useful information and return the dataframe

    :param feature: filename
    '''
    f = open(file_name) 
    data = json.load(f)

    df_list = []
    for restaurant, reviews in data.items():
        normalized_df = pd.json_normalize(reviews)
        normalized_df['restaurant'] = restaurant
        filtered_df = normalized_df.dropna(axis=1, how='all')
        df_list.append(filtered_df)

    return pd.concat(df_list, ignore_index=True)

# Load restaurant reviews data from web scraping
reviews = []
path = "{}/reviews".format(os.getcwd())
for file_name in os.listdir(path):
    with open(path+'/'+file_name, "r") as json_file:
        reviews.append(load_reviews(path+'/'+file_name))

df_reviews = pd.concat(reviews, ignore_index=True) 

In [3]:
df_reviews['photoCount'].fillna(0,inplace=True)
df_reviews.drop_duplicates()

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment.text,comment.language,restaurant
0,1012.0,321,NaN,11/2/2018,5,*Tartinery is one of the food vendors at Hudso...,en,tartinery-new-york-15
1,190.0,1421,NaN,8/6/2023,3,Overall the service is very slow. We ordered 2...,en,tartinery-new-york-15
2,117.0,294,2023.0,10/17/2022,4,Yum I love a place that has interesting drinks...,en,tartinery-new-york-15
3,154.0,171,2023.0,11/29/2021,5,I always look for a comfortable go to bar/rest...,en,tartinery-new-york-15
4,6668.0,2143,2023.0,4/6/2022,3,Tartinery is where you go for happy hour wine ...,en,tartinery-new-york-15
...,...,...,...,...,...,...,...,...
331688,301.0,77,2023.0,5/21/2022,4,Great brunch spot. Cute decor. The coffee was ...,en,brownstone-pancake-factory-edgewater
331689,324.0,114,2023.0,10/1/2022,4,An &#34;All American Dinner&#34; that is affor...,en,brownstone-pancake-factory-edgewater
331690,24.0,10,NaN,9/10/2023,5,Went today and got the triple d buffalo chicke...,en,brownstone-pancake-factory-edgewater
331691,5.0,35,NaN,5/7/2023,5,We came to Brownstone as a group of 30 on a bu...,en,brownstone-pancake-factory-edgewater


In [4]:
df_reviews

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment.text,comment.language,restaurant
0,1012.0,321,NaN,11/2/2018,5,*Tartinery is one of the food vendors at Hudso...,en,tartinery-new-york-15
1,190.0,1421,NaN,8/6/2023,3,Overall the service is very slow. We ordered 2...,en,tartinery-new-york-15
2,117.0,294,2023.0,10/17/2022,4,Yum I love a place that has interesting drinks...,en,tartinery-new-york-15
3,154.0,171,2023.0,11/29/2021,5,I always look for a comfortable go to bar/rest...,en,tartinery-new-york-15
4,6668.0,2143,2023.0,4/6/2022,3,Tartinery is where you go for happy hour wine ...,en,tartinery-new-york-15
...,...,...,...,...,...,...,...,...
331688,301.0,77,2023.0,5/21/2022,4,Great brunch spot. Cute decor. The coffee was ...,en,brownstone-pancake-factory-edgewater
331689,324.0,114,2023.0,10/1/2022,4,An &#34;All American Dinner&#34; that is affor...,en,brownstone-pancake-factory-edgewater
331690,24.0,10,NaN,9/10/2023,5,Went today and got the triple d buffalo chicke...,en,brownstone-pancake-factory-edgewater
331691,5.0,35,NaN,5/7/2023,5,We came to Brownstone as a group of 30 on a bu...,en,brownstone-pancake-factory-edgewater


In [5]:
#Transform dataframes to CSV files reusable across various analyses. 
current_directory = os.getcwd()
file_path = os.path.join(current_directory, 'restaurant_reviews_michelin_stars.csv')

df_reviews.to_csv(file_path, index=True)

# Restaurant Basics Data

N.B: The review data is complex to handle due to scale; since we pulled 50 reviews per Manhattan restaurant for ~10,877 properties, GitHub restrictions won't allow us to upload a > 100MG file. 

Instead, we scraped review data alphabetically (reviews for Masa goes into the M folder under one M file, Rubirosa to R under one R file, etc). 

Now, we clean this data and amalgmate ALL reviews into a dataframe and subsquent CSV. 

In [6]:
def load_restaurant_data(file_name):
    '''
    load restaurant data into pandas DataFrame from json file (web scraping)
    assign each restaurant its official name, return the dataframe

    :param feature: filename
    '''
    f = open(file_name) 
    data = json.load(f)
    df_list = []
    for restaurant, info in data.items():
        normalized_df = pd.json_normalize(info)
        normalized_df['restaurant'] = restaurant
        filtered_df = normalized_df.dropna(axis=1, how='all')
        df_list.append(filtered_df)

    # Concatenate all dataframes
    df_restaurants = pd.concat(df_list, ignore_index=True)
    return flatten_category(df_restaurants)


def extract_titles(row):
    '''
    extract and return all levels of data from a nested dictionary

    :param feature: 1 row of restaurant data
    '''
    # Extracting 'title' from each column if it is not None, otherwise using None
    return [row[i]['title'] if row[i] is not None else None for i in range(len(row))]

def flatten_category(df):
    '''
    extract all labels/categories of a restaurant, return the complete dataframe
    
    :param feature: pd dataframe
    '''
    new_category = pd.json_normalize(df['categories'])
    category_df = new_category.apply(extract_titles, axis=1, result_type='expand')

    # Renaming the columns
    category_df.columns = [f'category_{i}' for i in range(new_category.shape[1])]

    # Concatenate with the original dataframe if needed
    df = pd.concat([df, category_df], axis=1)
    return df

df_restaurants = load_restaurant_data('restaurants_michelin_stars.json')

In [7]:
michelin = json.load(open("michelin_alias_michelin_stars.json") )
df_restaurants['is_michelin'] = [1 if i in michelin.values() else 0 for i in df_restaurants['restaurant']]

In [8]:
# Exclude 8 brooklyn restaurants

'''
print(df_restaurants['is_michelin'].sum())
print(len(michelin.values()))

set_data = set(df_restaurants[df_restaurants['is_michelin']==1]['restaurant'])
set_michelin = set(michelin.values())

print(set_michelin-set_data) 
print(len(set_michelin-set_data))
print(len(set_michelin))
print(len(set_data))
'''

"\nprint(df_restaurants['is_michelin'].sum())\nprint(len(michelin.values()))\n\nset_data = set(df_restaurants[df_restaurants['is_michelin']==1]['restaurant'])\nset_michelin = set(michelin.values())\n\nprint(set_michelin-set_data) \nprint(len(set_michelin-set_data))\nprint(len(set_michelin))\nprint(len(set_data))\n"

In [9]:
df_restaurants.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.country,location.state,location.display_address,restaurant,location.address3,category_0,category_1,category_2,category_3,is_michelin
0,ce605OMECZ3jLZtk6dCDHA,le-pain-quotidien-new-york-137,Le Pain Quotidien,https://s3-media3.fl.yelpcdn.com/bphoto/z-YqDX...,False,https://www.yelp.com/biz/le-pain-quotidien-new...,84,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"[pickup, delivery]",...,US,NY,"[395 S End Ave, New York, NY 10280]",le-pain-quotidien-new-york-137,NaN,Coffee & Tea,Bakeries,Breakfast & Brunch,None,0
1,B8yFj4qt-HgMCbF5S-GkgA,creperie-new-york-6,Creperie,https://s3-media1.fl.yelpcdn.com/bphoto/uydGkc...,False,https://www.yelp.com/biz/creperie-new-york-6?a...,19,"[{'alias': 'creperies', 'title': 'Creperies'}]",2.5,[],...,US,NY,"[255 Liberty St, New York, NY 10281]",creperie-new-york-6,NaN,Creperies,None,None,None,0
2,WZLhPYaYSFy7M_-Jh1VuNw,dos-toros-taqueria-new-york-5,Dos Toros Taqueria,https://s3-media1.fl.yelpcdn.com/bphoto/fNdfBJ...,False,https://www.yelp.com/biz/dos-toros-taqueria-ne...,132,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,[],...,US,NY,"[225 Liberty St, Ste 242, New York, NY 10281]",dos-toros-taqueria-new-york-5,,Mexican,None,None,None,0
3,tFQRwk9SVOFgpMBZ55UrLw,umami-burger-new-york-17,Umami Burger,https://s3-media2.fl.yelpcdn.com/bphoto/uvquI3...,False,https://www.yelp.com/biz/umami-burger-new-york...,333,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.0,"[delivery, pickup]",...,US,NY,"[225 Liberty St, Ste 247, New York, NY 10281]",umami-burger-new-york-17,,Burgers,None,None,None,0
4,jhs35REmv8Yo9-Z27KQJSA,naya-brookfield-place-new-york-3,NAYA - Brookfield Place,https://s3-media3.fl.yelpcdn.com/bphoto/Zd6LgX...,False,https://www.yelp.com/biz/naya-brookfield-place...,32,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"[pickup, delivery]",...,US,NY,"[225 Liberty St, Fl 2, New York, NY 10281]",naya-brookfield-place-new-york-3,,Mediterranean,Lebanese,None,None,0


Now, we feature engineer to derive deeper insights & metrics.

In [10]:
df_reviews_stat = df_reviews[['restaurant', 'photoCount']].groupby('restaurant').count().rename(columns={'photoCount':'reviews_cnt'})
df_reviews_stat['elite_reviews_cnt'] = df_reviews[['restaurant', 'eliteYear']].groupby('restaurant').count()['eliteYear']
df_reviews_stat['elite_reviews_perc'] = df_reviews_stat['elite_reviews_cnt']/df_reviews_stat['reviews_cnt']
df_reviews_stat['user_photo_cnt_avg'] = df_reviews[['restaurant', 'photoCount']].groupby('restaurant').mean()['photoCount']
df_reviews_stat['user_review_cnt_avg'] = df_reviews[['restaurant', 'reviewCount']].groupby('restaurant').mean()['reviewCount']
df_reviews_stat['rating_avg'] = df_reviews[['restaurant', 'rating']].groupby('restaurant').mean()['rating']
df_reviews_stat['rating_med'] = df_reviews[['restaurant', 'rating']].groupby('restaurant').median()['rating']
df_reviews_stat['rating_std'] = df_reviews[['restaurant', 'rating']].groupby('restaurant').std()['rating']
df_reviews_stat['rating_std'] = df_reviews_stat['rating_std'].fillna(0) # if we only get one review, then the std should be 0
df_reviews_stat

,reviews_cnt,elite_reviews_cnt,elite_reviews_perc,user_photo_cnt_avg,user_review_cnt_avg,rating_avg,rating_med,rating_std
restaurant,,,,,,,,
1-bite-mediterranean-new-york,17,1,0.058824,36.176471,134.588235,3.058824,3.0,1.344925
1-chimi-sushi-new-york,15,2,0.133333,1983.400000,525.933333,2.933333,3.0,1.533747
1-columbia-deli-new-york,5,0,0.000000,2.800000,9.200000,1.800000,1.0,1.788854
1-dollar-pizza-new-york,50,14,0.280000,816.540000,304.920000,3.440000,4.0,1.514016
1-minute-cafe-new-york-2,1,0,0.000000,0.000000,8.000000,2.000000,2.0,0.000000
...,...,...,...,...,...,...,...,...
zzs-clam-bar-new-york,50,10,0.200000,633.340000,315.660000,4.260000,5.0,1.157231
çka-ka-qëllu-new-york-3,50,30,0.600000,740.680000,264.520000,4.520000,5.0,0.762381
épicerie-boulud-new-york-6,50,22,0.440000,910.640000,348.120000,3.760000,4.0,1.238168


In [11]:
df_final_restaurants = pd.merge(df_restaurants, df_reviews_stat, how='inner', on='restaurant')

In [12]:
df_final_restaurants

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,category_3,is_michelin,reviews_cnt,elite_reviews_cnt,elite_reviews_perc,user_photo_cnt_avg,user_review_cnt_avg,rating_avg,rating_med,rating_std
0,ce605OMECZ3jLZtk6dCDHA,le-pain-quotidien-new-york-137,Le Pain Quotidien,https://s3-media3.fl.yelpcdn.com/bphoto/z-YqDX...,False,https://www.yelp.com/biz/le-pain-quotidien-new...,84,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"[pickup, delivery]",...,None,0,50,16,0.320000,755.900000,199.860000,3.320000,4.0,1.531173
1,B8yFj4qt-HgMCbF5S-GkgA,creperie-new-york-6,Creperie,https://s3-media1.fl.yelpcdn.com/bphoto/uydGkc...,False,https://www.yelp.com/biz/creperie-new-york-6?a...,19,"[{'alias': 'creperies', 'title': 'Creperies'}]",2.5,[],...,None,0,19,1,0.052632,654.894737,244.842105,2.631579,2.0,1.300022
2,WZLhPYaYSFy7M_-Jh1VuNw,dos-toros-taqueria-new-york-5,Dos Toros Taqueria,https://s3-media1.fl.yelpcdn.com/bphoto/fNdfBJ...,False,https://www.yelp.com/biz/dos-toros-taqueria-ne...,132,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,[],...,None,0,50,24,0.480000,3183.000000,365.740000,4.040000,4.0,1.159873
3,tFQRwk9SVOFgpMBZ55UrLw,umami-burger-new-york-17,Umami Burger,https://s3-media2.fl.yelpcdn.com/bphoto/uvquI3...,False,https://www.yelp.com/biz/umami-burger-new-york...,333,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.0,"[delivery, pickup]",...,None,0,50,34,0.680000,941.800000,424.520000,3.240000,3.5,1.378553
4,jhs35REmv8Yo9-Z27KQJSA,naya-brookfield-place-new-york-3,NAYA - Brookfield Place,https://s3-media3.fl.yelpcdn.com/bphoto/Zd6LgX...,False,https://www.yelp.com/biz/naya-brookfield-place...,32,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"[pickup, delivery]",...,None,0,32,15,0.468750,603.875000,252.406250,4.250000,5.0,1.077632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10872,J0V2C6wAkHMLD9j7I50_ZQ,sgd-dubu-so-gong-dong-tofu-and-bbq-edgewater-2,SGD Dubu So Gong Dong Tofu & BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/xX4DnY...,False,https://www.yelp.com/biz/sgd-dubu-so-gong-dong...,228,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,[delivery],...,None,0,50,28,0.560000,979.220000,312.580000,3.860000,4.0,1.143036
10873,awvl9pC0RW8PT5A3NeLGMg,fuxingmei-sushi-house-edgewater,FuXingMei Sushi House,https://s3-media1.fl.yelpcdn.com/bphoto/ZgikoA...,False,https://www.yelp.com/biz/fuxingmei-sushi-house...,94,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"[delivery, pickup]",...,None,0,50,16,0.320000,609.660000,186.440000,4.000000,4.0,1.195229
10874,JmhZW6euOW_7L3E3v4fpKg,happy-wok-bronx,Happy Wok,,False,https://www.yelp.com/biz/happy-wok-bronx?adjus...,6,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.0,"[delivery, pickup]",...,None,0,6,0,0.000000,1.833333,5.833333,2.166667,1.0,1.834848
10875,S55v4pq633nTz7jbGmrCfg,justins-pizza-bronx,Justin's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/CH79G2...,False,https://www.yelp.com/biz/justins-pizza-bronx?a...,18,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"[delivery, pickup]",...,None,0,18,1,0.055556,3847.222222,312.888889,2.611111,2.0,1.685191


In [13]:
"""
CAVEAT: This CSV file is over 100MG; Github cannot support it, so we don't directly upload it. However, we use it in our analyses so if you are attempting to run code download this file by running the cells in this section. 
"""

current_directory = os.getcwd()
file_path = os.path.join(current_directory, 'restaurant_details_michelin_stars.csv')

df_final_restaurants.to_csv(file_path, index=True) 